<a href="https://colab.research.google.com/github/adnanmasood/AIF360/blob/master/examples/bias-removal-optim_preproc_adult.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://git@github.com/adnanmasood/AIF360.git

  Cloning https://****@github.com/bhomass/AIF360.git to /tmp/pip-req-build-igt74688
  Running command git clone -q 'https://****@github.com/bhomass/AIF360.git' /tmp/pip-req-build-igt74688
  Created wheel for aif360: filename=aif360-0.3.0-cp36-none-any.whl size=1226976 sha256=f7187afcfb557d8511fb8ddc5d9ebdac8864f1817f09783b9100c61f9beb1468
  Stored in directory: /tmp/pip-ephem-wheel-cache-utd7e75k/wheels/b0/cb/cf/e61262f367d6e3f0d65ae1e5cb7809b5d9952f1c0e64eaae42
Successfully built aif360


# Detecting and mitigating racial bias in income estimation 

it needs to know the attribute or attributes, called _protected attributes_, that are of interest: race is one example of a _protected attribute_ and age is a second.

First, the process starts with a _training dataset_, which contains a sequence of instances, where each instance has two components: the features and the correct prediction for those features.  Next, a machine learning algorithm is trained on this training dataset to produce a machine learning model.  This generated model can be used to make a prediction when given a new instance.  A second dataset with features and correct predictions, called a _test dataset_, is used to assess the accuracy of the model.

Since this test dataset is the same format as the training dataset, a set of instances of features and prediction pairs, often these two datasets derive from the same initial dataset.  A random partitioning algorithm is used to split the initial dataset into training and test datasets..

In [ ]:
import sys
sys.path.append("../")

import numpy as np
import pandas as pd

from aif360.metrics import BinaryLabelDatasetMetric

from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
            import load_preproc_data_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions\
            import get_distortion_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools

from IPython.display import Markdown, display

In [ ]:
np.random.seed(1)

### Step 2 Load dataset, specifying protected attribute, and split dataset into train and test
In Step 2 we load the initial dataset, setting the protected attribute to be race.  We then splits the original dataset into training and testing datasets.  A normal workflow would also use a test dataset for assessing the efficacy (accuracy, fairness, etc.) during the development of a machine learning model.  Finally, we set two variables (to be used in Step 3) for the privileged (1) and unprivileged (0) values for the race attribute.  These are key inputs for detecting and mitigating bias, which will be Step 3 and Step 4.  

In [ ]:
dataset_orig = load_preproc_data_adult(['race'])

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'race': 1}] # White
unprivileged_groups = [{'race': 0}] # Not white

### Step 3 Compute fairness metric on original training dataset
Now that we've identified the protected attribute 'race' and defined privileged and unprivileged values, we can detect bias in the dataset.  One simple test is to compare the percentage of favorable results for the privileged and unprivileged groups, subtracting the former percentage from the latter.   A negative value indicates less favorable outcomes for the unprivileged groups.  This is implemented in the method called mean_difference on the BinaryLabelDatasetMetric class.  The code below performs this check and displays the output:

In [ ]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.097036


### Step 4 Mitigate bias by transforming the original dataset
The previous step showed that the privileged group was getting 10.5% more positive outcomes in the training dataset.   Since this is not desirable, we are going to try to mitigate this bias in the training dataset. Optimized preprocessing algorithm will transform the dataset to have more equity in positive outcomes on the protected attribute for the privileged and unprivileged groups.

The algorithm requires some tuning parameters, which are set in the optim_options variable and passed as an argument along with some other parameters, including the 2 variables containg the unprivileged and privileged groups defined in Step 3.

We then call the fit and transform methods to perform the transformation, producing a newly transformed training dataset (dataset_transf_train).  Finally, we ensure alignment of features between the transformed and the original dataset to enable comparisons.

[1] Optimized Pre-Processing for Discrimination Prevention, NIPS 2017, Flavio Calmon, Dennis Wei, Bhanukiran Vinzamuri, Karthikeyan Natesan Ramamurthy, and Kush R. Varshney

In [ ]:
optim_options = {
    "distortion_fun": get_distortion_adult,
    "epsilon": 0.05,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.1, 0.05, 0]
}
    
OP = OptimPreproc(OptTools, optim_options)

OP = OP.fit(dataset_orig_train)
dataset_transf_train = OP.transform(dataset_orig_train, transform_Y=True)

dataset_transf_train = dataset_orig_train.align_datasets(dataset_transf_train)

print(dataset_transf_train)

Optimized Preprocessing: Objective converged to 0.000000
               instance weights            features  ...                     labels
                                protected attribute  ...                           
                                               race  ... Education Years=>12       
instance names                                       ...                           
391                         1.0                 0.0  ...                 0.0    0.0
1899                        1.0                 0.0  ...                 0.0    0.0
24506                       1.0                 1.0  ...                 1.0    1.0
32816                       1.0                 1.0  ...                 1.0    0.0
47892                       1.0                 1.0  ...                 1.0    0.0
...                         ...                 ...  ...                 ...    ...
4449                        1.0                 1.0  ...                 1.0    0.0
39024              

In [ ]:
# To export the dataset
#pd.set_option('display.max_rows', 50)
#from google.colab import drive
# Mount your Drive to the Colab VM.
#drive.mount('/gdrive')
#data = pd.DataFrame()
#data, _ = dataset_transf_train.convert_to_dataframe()
# Write the DataFrame to CSV file.
#with open('/gdrive/My Drive/output/dataset.csv', 'w') as f:
#  data.to_csv(f)
#print ("completed")


### Step 5 Compute fairness metric on transformed dataset
Now that we have a transformed dataset, we can check how effective it was in removing bias by using the same metric we used for the original training dataset in Step 3.  Once again, we use the function mean_difference in the BinaryLabelDatasetMetric class:

In [ ]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.050695


We see the mitigation step was very effective, the difference in mean outcomes is now -0.051074.  So we went from a 10.5% advantage for the privileged group to a 5.1% advantage for the privileged group &mdash; a reduction in more than half!

### Summary

As mentioned earlier, both fairness metrics and mitigation algorithms can be performed at various stages of the machine learning pipeline.  We recommend checking for bias as often as possible, using as many metrics are relevant for the application domain.  We also recommend incorporating bias detection in an automated continous integration pipeline to ensure bias awareness as a software project evolves.